In [1]:
import nltk
import numpy as np
import pandas as pd
import re

In [2]:
trainDf = pd.read_csv("train.tsv", sep="\t")
testDf = pd.read_csv("test.tsv", sep="\t")
subDf = pd.read_csv("sampleSubmission.csv")
trainDf.sample(5)

,PhraseId,SentenceId,Phrase,Sentiment
130998,130999,7060,a long patch of black ice,1
43498,43499,2102,developing any storytelling flow,2
141421,141422,7671,"the diverse , marvelously twisted shapes histo...",2
82934,82935,4282,infuriating film .,0
152050,152051,8296,is throwing up his hands in surrender,2


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=1, max_features=4500, stop_words='english')
trainBow = tfidf_vectorizer.fit_transform(trainDf['Phrase'])
trainBow.shape

(156060, 4500)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

X_train = trainBow
y_train = trainDf["Sentiment"]

In [5]:
# model = MultinomialNB(alpha=.1, fit_prior= True)

model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=1, max_features=4500, stop_words='english')
testBow = tfidf_vectorizer.fit_transform(testDf['Phrase'])
testBow.shape

(66292, 4500)

In [7]:
predictions = model.predict(testBow)
predictions

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [8]:
submission = pd.DataFrame({'PhraseId':subDf['PhraseId'],'Sentiment':predictions})

In [9]:
filename = 'moviePredictions.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: moviePredictions.csv
